In [1]:
# Import modules
import pandas as pd
import numpy as np
import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm
import matplotlib.pyplot as plt

print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
pd.set_option('display.max_colwidth',1000)

TensorFlow Version: 2.1.0
Hub version:  0.8.0


### Start here:

In [3]:
df = pd.read_csv('df_10K.csv')

In [4]:
df['breakthrough'].value_counts()

0.0    9000
1.0    1000
Name: breakthrough, dtype: int64

In [11]:
df = df[['text','breakthrough']]

In [12]:
df.head(5)

,text,breakthrough
0,"Systems and/or methods for supporting communications at a reduced bandwidth with a full bandwidth network such as a long-term evolution (LTE) network may be disclosed. For example, inband assignments such as downlink assignments and/or uplink grants may be provided and/or received and transmissions may be monitored and/or decoded based on the inband assignment. Additionally, information (e.g. a definition or configuration) associated with an ePDCCH may be provided and/or received and ePD -CCH resources may be monitored and/or decoded based on such information. An indication for support of a reduced bandwidth by the full bandwidth network may also be provided and/or received and control channels in the reduced or narrow bandwidth may be monitored and/or decoded based on the indication. A PRACH preamble and/or a multi-type sub frame definition may also be provided and/or used for support of such a reduced bandwidth.",1.0
1,"A sealing system configured to receive a container and seal the container with a lidding. The system comprises an in-feeder that is configured to receive a plurality of containers and transport a batch of containers that are selected from the plurality of containers; and a modular seal assembly that is configured to seal the batch of containers with a lidding, wherein the modular seal assembly comprises: a docking station that is configured to hold a plurality of sealing modules; and a sealing module that is configured to be removable from the docking station without removing any other one of the plurality of sealing modules.",1.0
2,"The present invention relates to an optical guide, its use in a lighting and / or signaling device equipping a motor vehicle, in particular a motor vehicle and its manufacturing process. It relates more particularly to an optical guide comprising a translucent polymer comprising nanoparticles, said nanoparticles permitting diffusion and / or emission of light.",1.0
3,"A surgical stapler can comprise an anvil comprising a staple pocket formed in a tissue contacting surface. The staple pocket can comprise a midline or centerline, a first forming cup, and a second forming cup. The first forming cup can comprise a first interior sidewall comprising a first vertical portion which is substantially perpendicular to the tissue contacting surface. The second forming cup can comprise a second interior sidewall comprising a second vertical portion which is substantially perpendicular to the tissue contacting surface. The first vertical portion and the second vertical portion can extend through the centerline, wherein the first interior sidewall and the second interior sidewall can comprise a trap for deforming a first staple leg of a staple to a first side of the centerline and for deforming a second staple leg of the staple to a second side of the centerline.",1.0
4,"The present invention provides a refrigerator, comprising: a cabinet that defines a first storage compartment; a first door that is opened or closed to allow or prevent access to an interior of the first storage compartment; a storing device coupled to the first door to define a second storage compartment, wherein, when the first door is closed, the second storage compartment is disposed in the first storage compartment; and a second door connected to the first door and configured to be opened or closed to allow or prevent access to an interior of the second storage compartment, wherein the interior of the second storage compartment is accessible when the first door is closed and the second door is opened, wherein the first door includes a plurality of first coupling parts to which the storing device is coupled, wherein the storing device includes: a frame; and a basket installed on the frame, wherein each of the frame and the basket includes at least one second coupling part inter...",1.0


In [13]:
df['text'] = df['text'].apply(str)

In [14]:
df['breakthrough'] = df['breakthrough'].astype(int)
df

,text,breakthrough
0,"Systems and/or methods for supporting communications at a reduced bandwidth with a full bandwidth network such as a long-term evolution (LTE) network may be disclosed. For example, inband assignments such as downlink assignments and/or uplink grants may be provided and/or received and transmissions may be monitored and/or decoded based on the inband assignment. Additionally, information (e.g. a definition or configuration) associated with an ePDCCH may be provided and/or received and ePD -CCH resources may be monitored and/or decoded based on such information. An indication for support of a reduced bandwidth by the full bandwidth network may also be provided and/or received and control channels in the reduced or narrow bandwidth may be monitored and/or decoded based on the indication. A PRACH preamble and/or a multi-type sub frame definition may also be provided and/or used for support of such a reduced bandwidth.",1
1,"A sealing system configured to receive a container and seal the container with a lidding. The system comprises an in-feeder that is configured to receive a plurality of containers and transport a batch of containers that are selected from the plurality of containers; and a modular seal assembly that is configured to seal the batch of containers with a lidding, wherein the modular seal assembly comprises: a docking station that is configured to hold a plurality of sealing modules; and a sealing module that is configured to be removable from the docking station without removing any other one of the plurality of sealing modules.",1
2,"The present invention relates to an optical guide, its use in a lighting and / or signaling device equipping a motor vehicle, in particular a motor vehicle and its manufacturing process. It relates more particularly to an optical guide comprising a translucent polymer comprising nanoparticles, said nanoparticles permitting diffusion and / or emission of light.",1
3,"A surgical stapler can comprise an anvil comprising a staple pocket formed in a tissue contacting surface. The staple pocket can comprise a midline or centerline, a first forming cup, and a second forming cup. The first forming cup can comprise a first interior sidewall comprising a first vertical portion which is substantially perpendicular to the tissue contacting surface. The second forming cup can comprise a second interior sidewall comprising a second vertical portion which is substantially perpendicular to the tissue contacting surface. The first vertical portion and the second vertical portion can extend through the centerline, wherein the first interior sidewall and the second interior sidewall can comprise a trap for deforming a first staple leg of a staple to a first side of the centerline and for deforming a second staple leg of the staple to a second side of the centerline.",1
4,"The present invention provides a refrigerator, comprising: a cabinet that defines a first storage compartment; a first door that is opened or closed to allow or prevent access to an interior of the first storage compartment; a storing device coupled to the first door to define a second storage compartment, wherein, when the first door is closed, the second storage compartment is disposed in the first storage compartment; and a second door connected to the first door and configured to be opened or closed to allow or prevent access to an interior of the second storage compartment, wherein the interior of the second storage compartment is accessible when the first door is closed and the second door is opened, wherein the first door includes a plurality of first coupling parts to which the storing device is coupled, wherein the storing device includes: a frame; and a basket installed on the frame, wherein each of the frame and the basket includes at least one second coupling part inter...",1
...,...,...
9995,"The disclosure relates to a pump unit (10, 110) which comprises a pump, in particular a liquid ring pump (11,

In [15]:
df.shape

(10000, 2)

In [16]:
df.apply(lambda x: sum(x.isnull()), axis=0)

text            0
breakthrough    0
dtype: int64

In [19]:
# Functions for constructing BERT Embeddings: input_ids, input_masks, input_segments and Inputs
MAX_SEQ_LEN=500 # max sequence length

def get_masks(tokens):
    """Masks: 1 for real tokens and 0 for paddings"""
    return [1]*len(tokens) + [0] * (MAX_SEQ_LEN - len(tokens))
 
def get_segments(tokens):
    """Segments: 0 for the first sequence, 1 for the second"""  
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (MAX_SEQ_LEN - len(tokens))

def get_ids(tokens, tokenizer):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (MAX_SEQ_LEN - len(token_ids))
    return input_ids

def create_single_input(sentence, tokenizer, max_len):
    """Create an input from a sentence"""
    stokens = tokenizer.tokenize(sentence)
    stokens = stokens[:max_len]
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
    ids = get_ids(stokens, tokenizer)
    masks = get_masks(stokens)
    segments = get_segments(stokens)

    return ids, masks, segments
 
def convert_sentences_to_features(sentences, tokenizer):
    """Convert sentences to features: input_ids, input_masks and input_segments"""
    input_ids, input_masks, input_segments = [], [], []
 
    for sentence in tqdm(sentences,position=0, leave=True):
      ids,masks,segments=create_single_input(sentence,tokenizer,MAX_SEQ_LEN-2)
      assert len(ids) == MAX_SEQ_LEN
      assert len(masks) == MAX_SEQ_LEN
      assert len(segments) == MAX_SEQ_LEN
      input_ids.append(ids)
      input_masks.append(masks)
      input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
          np.asarray(input_masks, dtype=np.int32), 
          np.asarray(input_segments, dtype=np.int32)]

def create_tonkenizer(bert_layer):
    """Instantiate Tokenizer with vocab"""
    vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case=bert_layer.resolved_object.do_lower_case.numpy() 
    tokenizer=bert.bert_tokenization.FullTokenizer(vocab_file,do_lower_case)
    return tokenizer

In [20]:
def bert_model(callable_object):
    # Load the pre-trained BERT base model
    bert_layer = hub.KerasLayer(handle=callable_object, trainable=True)  
   
    # BERT layer three inputs: ids, masks and segments
    input_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_ids")           
    input_masks = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_masks")       
    input_segments = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="segment_ids")
    
    inputs = [input_ids, input_masks, input_segments] # BERT inputs
    pooled_output, sequence_output = bert_layer(inputs) # BERT outputs
    
    # Add a hidden layer
    x = Dense(units=768, activation='relu')(pooled_output)
    x = Dropout(0.1)(x)
 
    # Add output layer
    outputs = Dense(2, activation="sigmoid")(x)

    # Construct a new model
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = bert_model("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [21]:
df.shape

(10000, 2)

In [22]:
# Create examples for training and testing
df = df.sample(frac=1) # Shuffle the dataset
tokenizer = create_tonkenizer(model.layers[3])
X_train = convert_sentences_to_features(df['text'][:8000], tokenizer)
X_test = convert_sentences_to_features(df['text'][8000:], tokenizer)

one_hot_encoded = to_categorical(df['breakthrough'].values)
y_train = one_hot_encoded[:8000]
y_test =  one_hot_encoded[8000:]

100%|██████████| 2000/2000 [00:04<00:00, 497.17it/s]


model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["acc"])

In [ ]:
# Train the model
BATCH_SIZE = 12
EPOCHS = 1

# Use Adam optimizer to minimize the categorical_crossentropy loss
opt = Adam(learning_rate=2e-5)
model.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['acc'])

# Fit the data to the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose = 1)

# Save the trained model
model.save('bert_model.h5')

Train on 8000 samples, validate on 2000 samples


# Load the pretrained nlp_model
from tensorflow.keras.models import load_model

new_model = load_model('bert_model.h5',custom_objects={'KerasLayer':hub.KerasLayer})

In [22]:
# Predict on test dataset
from sklearn.metrics import classification_report
pred_test = np.argmax(model.predict(X_test), axis=1)

In [23]:
print(classification_report(np.argmax(y_test,axis=1), pred_test))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1815
           1       0.00      0.00      0.00       185

    accuracy                           0.91      2000
   macro avg       0.45      0.50      0.48      2000
weighted avg       0.82      0.91      0.86      2000



/home/lin_menghsien/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
np.unique(pred_test, return_counts = True)

(array([0]), array([2000]))

In [25]:
# Predict on train dataset
from sklearn.metrics import classification_report
pred_train = np.argmax(model.predict(X_train), axis=1)

In [26]:
print(classification_report(np.argmax(y_train,axis=1), pred_train))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      7185
           1       0.00      0.00      0.00       815

    accuracy                           0.90      8000
   macro avg       0.45      0.50      0.47      8000
weighted avg       0.81      0.90      0.85      8000



In [24]:
model.save_weights('bert_weights.h5')

In [30]:
np.unique(pred_train, return_counts = True)

(array([0]), array([8000]))